# TimeSeriesForestRegressor

Regression counterpart of TS Forest: interval features + ensemble of trees.


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sktime.datasets import load_basic_motions, load_unit_test



In [ ]:
X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True)



In [ ]:
from sktime.regression.interval_based import TimeSeriesForestRegressor
from sklearn.metrics import mean_absolute_error

model = TimeSeriesForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, pred))
